In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 3, Finished, Available, Finished)

In [2]:
df_sales= pd.read_excel("abfss://1df115cc-23f0-4d1d-9a90-aad986d02161@onelake.dfs.fabric.microsoft.com/f79a512c-ee42-4057-bc18-ce8433d05b40/Files/Current/Sales*.xlsx",sheet_name="Sales")

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 4, Finished, Available, Finished)

In [3]:
df1=spark.createDataFrame(df_sales)

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 5, Finished, Available, Finished)

In [4]:
display(df1.head(10))

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 918e1154-1836-4994-92cc-5c968c25f96c)

In [5]:
df_Returns= pd.read_excel("abfss://1df115cc-23f0-4d1d-9a90-aad986d02161@onelake.dfs.fabric.microsoft.com/f79a512c-ee42-4057-bc18-ce8433d05b40/Files/Current/Sales*.xlsx",sheet_name="Returns")

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 7, Finished, Available, Finished)

In [6]:
df2=spark.createDataFrame(df_Returns)

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 8, Finished, Available, Finished)

In [7]:
display(df2.head(10))

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7cba937e-953b-4c07-8cde-2e3098544f9a)

In [8]:
df_final=df1.join(df2,df1.Order_ID==df2.Order_ID,how="left").drop(df2.Order_ID,df2.Customer_Name,df2.Sales_Amount)

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 10, Finished, Available, Finished)

In [9]:
df_mod=df_final.withColumns({"Order_Year": year("Order_Date"), \
                "Order_Month": month("Order_Date"), \
                "Created_TS": current_timestamp(),\
                "Modified_TS": current_timestamp(),\
                })

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 11, Finished, Available, Finished)

In [10]:
df_mod.createOrReplaceTempView("ViewSales")

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 12, Finished, Available, Finished)

In [11]:
%%sql
create table if not exists Project_Lakehouse.Bronze_Sales
(
    Order_ID	string,
    Order_Date	Date,
    Shipping_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	float,
    Quantity	float,
    Discount	 float,
    Profit 	 float,
    Shipping_Cost 	float,
    Order_Priority	string,
    Customer_ID	string,
    Customer_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
using DELTA
PARTITIONED by(Order_Year,Order_Month)

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
%%sql
Merge into  Project_Lakehouse.Bronze_Sales as BS
using ViewSales as VS
on  BS.Order_Year=VS.Order_Year and BS.Order_Month=VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then
update SET
BS.Order_Date	=	VS.Order_Date	,
BS.Shipping_Date	=	VS.Shipping_Date	,
BS.Aging	=	VS.Aging	,
BS.Ship_Mode	=	VS.Ship_Mode	,
BS.Product_Category	=	VS.Product_Category	,
BS.Product	=	VS.Product	,
BS.Sales 	=	VS.Sales 	,
BS.Quantity	=	VS.Quantity	,
BS.Discount	=	VS.Discount	,
BS.Profit 	=	VS.Profit 	,
BS.Shipping_Cost 	=	VS.Shipping_Cost 	,
BS.Order_Priority	=	VS.Order_Priority	,
BS.Customer_ID	=	VS.Customer_ID	,
BS.Customer_Name	=	VS.Customer_Name	,
BS.Segment	=	VS.Segment	,
BS.City	=	VS.City	,
BS.State	=	VS.State	,
BS.Country	=	VS.Country	,
BS.Region  	=	VS.Region  	,
BS.Return  	=	VS.Return  	,
BS.Modified_TS	=	VS.Modified_TS

when not matched then
INSERT
(
BS.Order_ID,
BS.Order_Date	,
BS.Shipping_Date	,
BS.Aging	,
BS.Ship_Mode	,
BS.Product_Category	,
BS.Product	,
BS.Sales 	,
BS.Quantity	,
BS.Discount	,
BS.Profit 	,
BS.Shipping_Cost 	,
BS.Order_Priority	,
BS.Customer_ID	,
BS.Customer_Name	,
BS.Segment	,
BS.City	,
BS.State	,
BS.Country	,
BS.Region  	,
BS.Return,
BS.Order_Year  	,
BS.Order_Month  	,
BS.Created_TS   	,
BS.Modified_TS
)
values
(
VS.Order_ID,
VS.Order_Date	,
VS.Shipping_Date	,
VS.Aging	,
VS.Ship_Mode	,
VS.Product_Category	,
VS.Product	,
VS.Sales 	,
VS.Quantity	,
VS.Discount	,
VS.Profit 	,
VS.Shipping_Cost 	,
VS.Order_Priority	,
VS.Customer_ID	,
VS.Customer_Name	,
VS.Segment	,
VS.City	,
VS.State	,
VS.Country	,
VS.Region  	,
VS.Return,
VS.Order_Year  	,
VS.Order_Month  	,
VS.Created_TS   	,
VS.Modified_TS

)

StatementMeta(, 35167c0a-3548-4aa2-ab16-f97c953c68e5, 14, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>